In [1]:
#surrogate models
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split

data_name = '10240'

data_file = f'../{data_name}_vectors_drags_lifts.csv'
df = TabularDataset(data_file)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=777)

#exclue the first two columns of train data
train_data = train_df.drop(columns=['i', 'name'])
train_data.head()

/home/xli/anaconda3/envs/surrogate_autogluon/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_10233,dim_10234,dim_10235,dim_10236,dim_10237,dim_10238,dim_10239,dim_10240,drag,lift
406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.395,0.425
54,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.645,0.885
241,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.446,0.756
952,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.384,0.177
337,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.381,0.074


In [2]:
label = 'drag'
print("Summary of class variable: \n", train_data[label].describe())

Summary of class variable: 
 count    837.000000
mean       0.445233
std        0.112500
min        0.051000
25%        0.369000
50%        0.427000
75%        0.513000
max        0.944000
Name: drag, dtype: float64


In [3]:
import os
save_path = f'./agModels-{data_name}_{label}'  # specifies folder to store trained models
if not os.path.exists(save_path):
    os.makedirs(save_path)

bag_folds = 5 #suggestion range [5, 10]
bag_sets = 3 #suggestion range [1, 20]
stack_levels = 3 #suggestion range [0, 3]
metric = 'root_mean_squared_error' #Regression:mean_absolute_error, mean_squared_error,root_mean_squared_error (default), r2
predictor = TabularPredictor(label=label, path=save_path, eval_metric=metric).fit(train_data, 
                                                                                  presets='best_quality', 
                                                                                  auto_stack="True", 
                                                                                  num_bag_folds=bag_folds, 
                                                                                  num_bag_sets=bag_sets,
                                                                                  num_stack_levels=stack_levels,
                                                                                  verbosity=4)

Presets specified: ['best_quality']
============ fit kwarg info ============
User Specified kwargs:
{'auto_stack': 'True',
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'verbosity': 4}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': None,
 'auto_stack': 'True',
 'calibrate': 'auto',
 'excluded_model_types': None,
 'feature_generator': 'auto',
 'feature_prune_kwargs': None,
 'holdout_frac': None,
 'hyperparameter_tune_kwargs': None,
 'keep_only_best': False,
 'name_suffix': None,
 'num_bag_folds': 5,
 'num_bag_sets': 3,
 'num_stack_levels': 3,
 'pseudo_data': None,
 'refit_full': False,
 'save_space': False,
 'set_best_to_refit_full': False,
 'unlabeled_data': None,
 'use_bag_holdout': False,
 'verbosity': 4}
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=5, num_bag_sets=3
	Consider setting `time_limit` to ensure training finishes within an expected durat

In [4]:
test_data = test_df.drop(columns=['i', 'name'])
# val_data.head()
y_val = test_data[label]
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,dim_1,dim_2,dim_3,dim_4,dim_5,dim_6,dim_7,dim_8,dim_9,dim_10,...,dim_10232,dim_10233,dim_10234,dim_10235,dim_10236,dim_10237,dim_10238,dim_10239,dim_10240,lift
1046,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.346
657,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.654
127,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.052
951,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.604
134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.199


In [5]:
# %%capture log_output
# %config InlineBackend.figure_format = 'retina'
# %config Application.log_level = 'DEBUG'
# %config IPCompleter.greedy = True

predictor = TabularPredictor.load(save_path)  # unnecessary, just demonstrates how to load previously-trained predictor from file
y_pred = predictor.predict(test_data_nolab)
for item in y_pred:
    print(item)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_val, y_pred=y_pred, auxiliary_metrics=True)
print(perf)

results = predictor.fit_summary(show_plot=True)
print(results)
print(predictor.leaderboard(test_data, silent=True))

# with open('./output_all_parts.log', 'w') as f:
#     f.write(log_output.stdout)

Loading: ./agModels-10240_drag/predictor.pkl
Loading: ./agModels-10240_drag/learner.pkl
Loading: ./agModels-10240_drag/models/trainer.pkl
Loading: ./agModels-10240_drag/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/ExtraTreesMSE_BAG_L2/model.pkl
Loading: ./agModels-10240_drag/models/NeuralNe

0.4793897271156311
0.3007722496986389
0.42282944917678833
0.4341621994972229
0.49117815494537354
0.39879125356674194
0.4378107786178589
0.3628047704696655
0.41072380542755127
0.3959246575832367
0.5728473663330078
0.5650310516357422
0.4029695391654968
0.3273037374019623
0.5267558097839355
0.5489460229873657
0.3461034595966339
0.5235460996627808
0.5927423238754272
0.541691780090332
0.4638623297214508
0.39146190881729126
0.6315969228744507
0.4435417354106903
0.41047683358192444
0.4012417793273926
0.5629673600196838
0.42277267575263977
0.5523302555084229
0.3400198221206665
0.40177586674690247
0.44647225737571716
0.4318239688873291
0.4008459448814392
0.42748910188674927
0.35207271575927734
0.44065752625465393
0.38074856996536255
0.3369844853878021
0.40347009897232056
0.452187180519104
0.3473742604255676
0.6252174973487854
0.41470929980278015
0.554286003112793
0.38625937700271606
0.35688698291778564
0.5023987293243408
0.4051309823989868
0.32650843262672424
0.558217465877533
0.460876166820526

Loading: ./agModels-10240_drag/models/NeuralNetFastAI_BAG_L2/model.pkl
Loading: ./agModels-10240_drag/models/XGBoost_BAG_L2/model.pkl
Loading: ./agModels-10240_drag/models/NeuralNetTorch_BAG_L2/model.pkl
Loading: ./agModels-10240_drag/models/LightGBMLarge_BAG_L2/model.pkl
Loading: ./agModels-10240_drag/models/WeightedEnsemble_L3/model.pkl
Loading: ./agModels-10240_drag/models/LightGBMXT_BAG_L3/model.pkl
Loading: ./agModels-10240_drag/models/LightGBM_BAG_L3/model.pkl
Loading: ./agModels-10240_drag/models/RandomForestMSE_BAG_L3/model.pkl
Loading: ./agModels-10240_drag/models/CatBoost_BAG_L3/model.pkl
Loading: ./agModels-10240_drag/models/ExtraTreesMSE_BAG_L3/model.pkl
Loading: ./agModels-10240_drag/models/NeuralNetFastAI_BAG_L3/model.pkl
Loading: ./agModels-10240_drag/models/XGBoost_BAG_L3/model.pkl
Loading: ./agModels-10240_drag/models/NeuralNetTorch_BAG_L3/model.pkl
Loading: ./agModels-10240_drag/models/LightGBMLarge_BAG_L3/model.pkl
Loading: ./agModels-10240_drag/models/WeightedEnsemb

*** Summary of fit() ***
Estimated performance of each model:
                     model  score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      WeightedEnsemble_L3  -0.071589      42.697449  2098.714436                0.001031           0.609406            3       True         22
1      WeightedEnsemble_L4  -0.071732      74.569921  3184.738425                0.001384           0.669107            4       True         32
2      WeightedEnsemble_L2  -0.071926       6.589473   272.346195                0.001471           0.648966            2       True         12
3   RandomForestMSE_BAG_L2  -0.072699      34.030623  1960.208445                3.396782          23.923739            2       True         15
4      WeightedEnsemble_L5  -0.072829     117.663265  4272.361269                0.001061           0.434959            5       True         42
5     LightGBMLarge_BAG_L3  -0.072920      65.060601  3017.463984         

/home/xli/anaconda3/envs/surrogate_autogluon/lib/python3.10/site-packages/autogluon/core/utils/plots.py:138: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')
Loading: ./agModels-10240_drag/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/XGBoost_BAG_L1/model.pkl

                     model  score_test  score_val  pred_time_test  \
0      WeightedEnsemble_L2   -0.073054  -0.071926        8.063035   
1          LightGBM_BAG_L2   -0.073106  -0.074753       24.558772   
2      WeightedEnsemble_L5   -0.073194  -0.072829       72.305431   
3          CatBoost_BAG_L1   -0.073203  -0.074600        8.992277   
4      WeightedEnsemble_L3   -0.073451  -0.071589       25.809273   
5   NeuralNetFastAI_BAG_L4   -0.073683  -0.075666       68.064857   
6          CatBoost_BAG_L2   -0.073764  -0.073016       31.127023   
7          LightGBM_BAG_L1   -0.073879  -0.074349        2.546667   
8          CatBoost_BAG_L3   -0.073945  -0.073334       54.078601   
9        LightGBMXT_BAG_L3   -0.074046  -0.074433       47.492467   
10  NeuralNetFastAI_BAG_L3   -0.074078  -0.074981       46.268884   
11  RandomForestMSE_BAG_L4   -0.074083  -0.074169       66.869008   
12  RandomForestMSE_BAG_L2   -0.074112  -0.072699       22.041851   
13  NeuralNetFastAI_BAG_L2   -0.07

In [6]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon identified the following types of features:")
print(predictor.feature_metadata)

AutoGluon infers problem type is:  regression
AutoGluon identified the following types of features:
('float', [])     :    1 | ['lift']
('int', ['bool']) : 9783 | ['dim_1', 'dim_2', 'dim_3', 'dim_4', 'dim_5', ...]


In [7]:
train_data_pred = predictor.predict(train_data)
test_data_pred = predictor.predict(test_data)

import numpy as np
#save np array y_train_hat to a csv file
np.savetxt(f'./{data_name}_vectors_y_test_hat_{label}.csv', test_data_pred, delimiter=',')
np.savetxt(f'./{data_name}_vectors_y_train_hat_{label}.csv', train_data_pred, delimiter=',')

Loading: ./agModels-10240_drag/models/CatBoost_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/ExtraTreesMSE_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/KNeighborsDist_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/KNeighborsUnif_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/LightGBMLarge_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/LightGBMXT_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/LightGBM_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/NeuralNetFastAI_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/NeuralNetTorch_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/RandomForestMSE_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/XGBoost_BAG_L1/model.pkl
Loading: ./agModels-10240_drag/models/ExtraTreesMSE_BAG_L2/model.pkl
Loading: ./agModels-10240_drag/models/NeuralNetFastAI_BAG_L2/model.pkl
Loading: ./agModels-10240_drag/models/NeuralNetTorch_BAG_L2/model.pkl
Loading: ./agModels-10240_drag/models/Rando